# A New Class of Compact Formulations for Vehicle Routing Problems 

First step is to install the necassary packages

In [2]:
using JuMP                # For mathematical modeling
using GLPK                # For the free GLPK solver
using Gurobi
using MathOptInterface    # For solver compatibility and optimization interfaces
MOI = MathOptInterface
using Random
const GRB_ENV = Gurobi.Env()


Set parameter Username
Set parameter LicenseID to value 2605064
Academic license - for non-commercial use only - expires 2026-01-04


Gurobi.Env(Ptr{Nothing} @0x000000012d9c5600, false, 0)

# Instructions
You should run all of the code from the start in the correct order. Structs that are basic are compiled first, and second the ones that are dependent on those are build. The only thing that needs to be changed is the file path of the solomon instances that are freely available online. 

## Create an instance
- Create data structures for the VRPTW
- Read Solomon instances
- And create some basic functions

In [3]:
number_of_customers = 50
inst = "r101"

"r101"

- Path is a stuct that stores the "LA arcs" as paths from a starting customer to the end customer. Every customer has a list of paths for its neighborhood. 

In [4]:
mutable struct Path
    id::Int
    path::Vector{Int}
    future::Vector{Int}
    load::Float32
    time::Float32
end

In [5]:
mutable struct Demand_node
    id::Int                      
    customer::Int
    buck_inx::Int
    min_load::Float32
    max_load::Float32
    forward::Vector{Int}
    backward::Vector{Int}
end

In [6]:
mutable struct Time_node
    id::Int                      
    customer::Int
    buck_inx::Int
    min_time::Float32
    max_time::Float32
    forward::Vector{Int}
    backward::Vector{Int}
end

In [7]:
mutable struct Solution
    demand_nodes::Vector{Demand_node}
    time_nodes::Vector{Time_node}
    customers::Vector{Any}
    paths::Vector{Path}
end

- all_paths is a list of all paths for all customers. It is initiated below as empty. When a Path is created it inmediately gets added to the list of all paths with a unique id that equals the index of the list of all_paths for easy access. 

- Customer is a struct that stores information for every customer, id, coordinates, time windows, demand, service time, and buckets for resources, including the set of paths for its neigborhood. 

In [8]:
mutable struct Customer
    id::Int                      # Customer ID
    x::Float32                   # X-coordinate
    y::Float32                   # Y-coordinate
    time_window::Tuple{Int, Int} # (start time, end time)
    demand::Int                  # Demand at the customer location
    service_time::Int            # Time spent at the customer location
    LA::Vector{Int}                 # Set of customer IDs, initially empty
    W_D::Vector{Float64} # bucket of thresholds
    W_D_remove::Vector{Float64} # tempodary to remember what to remove from W_D
    W_T::Vector{Float64} # bucket of time
    W_T_remove::Vector{Float64}
    B_D::Vector{Demand_node} # holds nodes in the demand Graph
    B_T::Vector{Time_node}
    paths::Vector{Path}
    N::Vector{Int} # Dymanic set to contract
    Not_N::Vector{Int}
    k::Int
end

In [9]:

# Define Depot struct (with service time)
mutable struct Depot
    id::Int
    x::Float32
    y::Float32
    time_window::Tuple{Int, Int}
    demand::Int
    service_time::Int  # Time spent at the depot
    B_D::Vector{Demand_node}
    B_T::Vector{Time_node}
end

### Default initiation
- ns is the size of the la neighbors, ds is the step in the demand bucket, and ts is the step in the time bucket.

In [10]:
ns = 6
ds = 50
ts = 50

50

### global all paths

In [11]:
all_paths = Vector{Path}()

Path[]

#### Function that reads Solomon instances

In [12]:
function read_vrptw_instance(file_path::String)
    customers = []
    depot = nothing
    vehicle_capacity = 0  # Placeholder for vehicle capacity
    number_of_vehicles = 0  # Placeholder for number of vehicles

    open(file_path, "r") do io
        for (line_num, line) in enumerate(eachline(io))
            # Extract vehicle information from the header
            if line_num == 5
                data = split(strip(line))
                number_of_vehicles = parse(Int, data[1])  # Number of vehicles
                vehicle_capacity = parse(Int, data[end])  # Vehicle capacity
            end
            
            # Skip the header
            if line_num <= 9
                continue
            end

            # Parse data line
            data = split(strip(line))
            if length(data) == 7
                id = parse(Int, data[1])
                x = parse(Float32, data[2])
                y = parse(Float32, data[3])
                demand = parse(Int, data[4])
                ready_time = parse(Int, data[5])
                due_time = parse(Int, data[6])
                service_time = parse(Int, data[7])

                if id == 0
                    # Create depot
                    depot = Depot((number_of_customers+1), x, y, (ready_time, due_time), 0, service_time,[],[])
                else
                    # Create customer
                    push!(customers, Customer(id, x, y, (ready_time, due_time), demand, service_time,[], [], [],[],[],[],[],[],[],[],ns))
                end
            end
            if length(customers) == number_of_customers
                break
            end
        end
    end

    if isnothing(depot)
        error("No depot found in file!")
    end
    all_locations = vcat(customers, [depot])
    return all_locations,vehicle_capacity, number_of_vehicles
end


read_vrptw_instance (generic function with 1 method)

### change this file path you your txt file of the solomon instace that you want to run.

In [13]:
file_path = "/Users/simulation/Desktop/optym/In/"* inst* ".txt"

"/Users/simulation/Desktop/optym/In/r101.txt"

- read the instace and store information about all the customers, vehicle capacity.

In [14]:

all_locations, vehicle_capacity, number_of_vehicles = read_vrptw_instance(file_path)
num_locations = length(all_locations)  # Number of customers + depot
max_time =  all_locations[end].time_window[2]
vehicle_capacity


200

#### We define a function that returns the distance between two customers. 
- The distance is rounded down to consider only the first decimal of the distance to get the same results as in the paper 

In [15]:
function distance(x1::Float32, y1::Float32, x2::Float32, y2::Float32)
    d = sqrt((x1 - x2)^2 + (y1 - y2)^2)
    truncated_d = floor(d * 10) / 10 
    return truncated_d
end

distance (generic function with 1 method)

- easier way to get the distance from two customers with out typing every time the coordinates. 

In [16]:
function distance_2(cust1::Int, cust2::Int)
    return distance(all_locations[cust1].x, all_locations[cust1].y,all_locations[cust2].x, all_locations[cust2].y)
end

distance_2 (generic function with 1 method)

In [17]:
function distance_3(cust1::Int, cust2::Int)
    return distance_2(cust1::Int, cust2::Int) + all_locations[cust1].service_time
end

distance_3 (generic function with 1 method)

### Preprocessing
-tighten the time windows. The distance from the depot to arrive to the customer since the triangle inequality holds, for the vehicle routing problem. 

In [18]:
## Tighten time windows
for u in all_locations
    minim = distance_3(all_locations[end].id, u.id)
    minim = Int(ceil(minim))
    early_t = u.time_window[1]

    if early_t < minim   
        println("early time window is too loose ",  minim, " " ,early_t, " customer = ", u.id)
        early_t = Int(ceil(minim))
    end
    maxim = distance_3(u.id, all_locations[end].id)
    late_t = u.time_window[2]
    maxim += late_t
    if maxim > all_locations[end].time_window[2] ## the time to go to the depot is not enought
        maxim = maxim - all_locations[end].time_window[2] ## the excess from the bound
        late_t = late_t - maxim ## late_t has to be reduced at least by the difference.
        late_t = Int(floor(late_t))
        println("late time window is too loose at the end ",  late_t , " " , u.time_window[2], " customer = ", u.id)
    end
    u.time_window = (early_t,late_t)
end


early time window is too loose 42 41 customer = 36


## New Compact formulation
- First we create the buckets and necessary data structures
- Next we build functions
- Finally, we build the algorithm

### Initiate the set of LA neighbours 

- is_feasible is a function used to create the LA neighbors of a customer by making sure it is feasible to go from the depot to customer 1 and next to customer 2 without violating any resource constraints.

In [19]:
function is_feasible(cust1::Int, cust2::Int)
    time = max(all_locations[cust1].time_window[1], distance_3(cust1,length(all_locations)))
    time += distance_3(cust1,cust2)
    one = (all_locations[cust2].time_window[2]>= time)
    demand = all_locations[cust2].demand
    demand += all_locations[cust1].demand
    two =  (demand <= vehicle_capacity)
    return (one && two)
end

is_feasible (generic function with 1 method)

- Next loop builds all the set of nearest neigbors for all customers

In [20]:
for i in 1:(num_locations - 1)
    cus = all_locations[i]  # Current customer
    customer_ids = [c.id for c in all_locations[1:end - 1] if c.id != cus.id]  # Exclude `cus` ID
    # Sort customer IDs by distance to `cus`
    nearest_n = sort(customer_ids, by = cid -> distance(cus.x, cus.y, all_locations[cid].x, all_locations[cid].y))
    filter!(j -> is_feasible(i, j), nearest_n)
    # Take the top `LA_size` customers
    nearest_n = nearest_n[1:min(ns, length(nearest_n))]
    all_locations[i].LA = nearest_n
    all_locations[i].N = nearest_n
end

### function that up dates set of customers not in $N$

In [21]:
function update_not_N(customer::Int)
    not_N = [n for n in num_locations if !(n in all_locations[customer].N) && n!=customer]
    all_locations[customer].Not_N = not_N
end
    

update_not_N (generic function with 1 method)

In [22]:
for i in num_locations-1
    update_not_N(i)
end

### Initiate $W^D$

In [23]:
for i in 1:(num_locations - 1)
    all_locations[i].W_D = []
    demand = all_locations[i].demand
    last_d = all_locations[i].demand
    while last_d <= vehicle_capacity
        last_d = min(vehicle_capacity, last_d)
        push!(all_locations[i].W_D,last_d)
        last_d += ds
    end
end
all_demand_nodes =[]


Any[]

In [24]:
function update_demand_edges()
    ##############################################################################################
    # for every customer add a demand node based on the thresholds W_D. 
    # HEre we are just creating the nodes
    # and not connecting them 
    for u in 1:(num_locations-1)
        d_plus = all_locations[u].W_D[1]
        id = length(all_demand_nodes) + 1
        push!(all_demand_nodes, Demand_node(id,u,1, all_locations[u].demand , d_plus,[],[]))
        push!(all_locations[u].B_D, all_demand_nodes[end])
        for d in 2:length(all_locations[u].W_D)
            id = length(all_demand_nodes) + 1
            d_plus = all_locations[u].W_D[d]
            d_minus = all_locations[u].W_D[d-1]+1
            push!(all_demand_nodes, Demand_node(id, u, d, d_minus,d_plus,[],[]))
            push!(all_locations[u].B_D, all_demand_nodes[end])
        
        end
    end

    push!(all_demand_nodes, Demand_node(length(all_demand_nodes)+1, num_locations, 0, 0, 0,[],[])) #depot_demand_node
    push!(all_locations[end].B_D, all_demand_nodes[end]) ## add the depot node to the depot. 
    ##############################################################################################
    
    ##############################################################################################
    ## connect every node with the same customer to the immediate one that follows. 
    ## no need to connect with the depot yet just connetc the customer node with the next
    for u in 1:num_locations-1
        for B_u in 1:length(all_locations[u].B_D)-1
            push!(all_locations[u].B_D[B_u].forward, all_locations[u].B_D[B_u+1].id)
            push!(all_locations[u].B_D[B_u+1].backward, all_locations[u].B_D[B_u].id)
        end
    end
    ##############################################################################################
    ## connect the demand node from one customer to the demand node of another customer.
    ## here again we exclude the depot
    for u in 1:num_locations-1
        for B_u in 1:length(all_locations[u].B_D)
            for w in 1:num_locations -1
                if u != w
                    for B_w in 1:length(all_locations[w].B_D)
                        if all_locations[w].B_D[B_w].max_load >= all_locations[u].B_D[B_u].min_load + all_locations[w].demand
                            push!(all_locations[u].B_D[B_u].forward, all_locations[w].B_D[B_w].id)
                            push!(all_locations[w].B_D[B_w].backward, all_locations[u].B_D[B_u].id)
                            break
                        end
                    end
                end
            end
        end
        ##############################################################################################
        ## connect the last and the first demand node from each customer u to the depot demand node.
        push!(all_locations[u].B_D[end].forward, length(all_demand_nodes))
        push!(all_locations[u].B_D[1].backward, length(all_demand_nodes))
        push!(all_demand_nodes[end].forward,all_locations[u].B_D[1].id)
        push!(all_demand_nodes[end].backward, all_locations[u].B_D[end].id )
        ##############################################################################################
    end
    return all_demand_nodes
end

update_demand_edges (generic function with 1 method)

In [25]:
function update_demand_edges(sol::Solution)
    ##############################################################################################
    # reset the buckets list to empty and rebuild them again at the later step.
    # this could be improved in the future by only eliminating nodes that need to be removed and 
    # keeping nodes that need to stay.
    sol.demand_nodes = []
    
    for u in 1:(num_locations)
        sol.customers[u].B_D = []
    end
    
    ##############################################################################################
    # for every customer add a demand node based on the thresholds W_D. 
    # HEre we are just creating the nodes
    # and not connecting them 
    for u in 1:(num_locations-1)
        d_plus = sol.customers[u].W_D[1]
        id = length(sol.demand_nodes) + 1
        push!(sol.demand_nodes, Demand_node(id, u, 1, sol.customers[u].demand, d_plus, [], []))
        push!(sol.customers[u].B_D, sol.demand_nodes[end])
        for d in 2:length(sol.customers[u].W_D)
            id = length(sol.demand_nodes) + 1
            d_plus = sol.customers[u].W_D[d]
            d_minus = sol.customers[u].W_D[d-1] + 1
            push!(sol.demand_nodes, Demand_node(id, u, d, d_minus, d_plus, [], []))
            push!(sol.customers[u].B_D, sol.demand_nodes[end])
        end
    end

    push!(sol.demand_nodes, Demand_node(length(sol.demand_nodes) + 1, num_locations, 0, 0, 0, [], [])) #depot_time_node
    push!(sol.customers[end].B_D, sol.demand_nodes[end]) ## add the depot node to the depot.
    ##############################################################################################
    
    ##############################################################################################
    ## connect every node with the same customer to the immediate one that follows. 
    ## no need to connect with the depot yet just connect the customer node with the next

    for u in 1:num_locations - 1
        for B_u in 1:length(sol.customers[u].B_D) - 1
            push!(sol.customers[u].B_D[B_u].forward, sol.customers[u].B_D[B_u + 1].id)
            push!(sol.customers[u].B_D[B_u + 1].backward, sol.customers[u].B_D[B_u].id)
        end
    end
    ##############################################################################################
    ## connect the demand node from one customer to the demand node of another customer.
    ## here again we exclude the depot
    
    for u in 1:num_locations - 1
        for B_u in 1:length(sol.customers[u].B_D)
            for w in 1:num_locations - 1
                if u != w
                    for B_w in 1:length(sol.customers[w].B_D)
                        if sol.customers[w].B_D[B_w].max_load >= sol.customers[u].B_D[B_u].min_load + sol.customers[w].demand
                            push!(sol.customers[u].B_D[B_u].forward, sol.customers[w].B_D[B_w].id)
                            push!(sol.customers[w].B_D[B_w].backward, sol.customers[u].B_D[B_u].id)
                            break
                        end
                    end
                end
            end
        end
        
        ##############################################################################################
        ## connect the last and the first demand node from each customer u to the depot demand node.
        push!(sol.customers[u].B_D[end].forward, length(sol.demand_nodes))
        push!(sol.customers[u].B_D[1].backward, length(sol.demand_nodes))
        push!(sol.demand_nodes[end].forward, sol.customers[u].B_D[1].id)
        push!(sol.demand_nodes[end].backward, sol.customers[u].B_D[end].id)
        
        ##############################################################################################
        if sol.customers[end].B_D[1].id != length(sol.demand_nodes)
            println(" the nodes did not update correctly and depot is still the old depot ")
            println("length ", length(sol.customers[end].B_D))
        end
    end
end


update_demand_edges (generic function with 2 methods)

### Initiate $W^T$

In [26]:
for i in 1:(num_locations - 1)
    early_t = all_locations[i].time_window[1]
    late_t = all_locations[i].time_window[2]
    all_locations[i].W_T = []
    threshold = early_t
    threshold = min(late_t, threshold )
    push!(all_locations[i].W_T, threshold)
    while threshold < late_t
        threshold += ts
        threshold = min(late_t, threshold)
        push!(all_locations[i].W_T, threshold)
        
    end
end
all_time_nodes = []

Any[]

In [27]:
function update_time_edges()
    ##############################################################################################
    # for every customer add a time node based on the thresholds W_T. 
    # HEre we are just creating the nodes
    # and not connecting them 
    for u in 1:(num_locations-1)
        t_plus = all_locations[u].W_T[1]
        id = length(all_time_nodes) + 1
        push!(all_time_nodes, Time_node(id, u, 1, all_locations[u].time_window[1], t_plus,[],[]))
        push!(all_locations[u].B_T, all_time_nodes[end])
        for t in 2:length(all_locations[u].W_T)
            id = length(all_time_nodes) + 1
            t_plus = all_locations[u].W_T[t]
            t_minus = all_locations[u].W_T[t-1]+1
            push!(all_time_nodes, Time_node(id, u, t, t_minus,t_plus,[],[]))
            push!(all_locations[u].B_T, all_time_nodes[end])
        end
    end
    
    push!(all_time_nodes, Time_node(length(all_time_nodes) + 1, num_locations, 0, 0, all_locations[end].time_window[2], [],[]))
    push!(all_locations[end].B_T, all_time_nodes[end]) ## add the depot node to the depot. 
    ##############################################################################################
    
    ##############################################################################################
    ## connect every node with the same customer to the immediate one that follows. 
    ## no need to connect with the depot yet just connect the customer node with the next
    for u in 1:num_locations-1
        for B_u in 1:length(all_locations[u].B_T)-1
            push!(all_locations[u].B_T[B_u].forward, all_locations[u].B_T[B_u+1].id)
            push!(all_locations[u].B_T[B_u+1].backward, all_locations[u].B_T[B_u].id)
        end
    end
    ##############################################################################################
    ## connect the time node from one customer to the time node of another customer.
    ## here again we exclude the depot
    for u in 1:num_locations-1
        for B_u in 1:length(all_locations[u].B_T)
            for w in 1:num_locations -1
                if u != w
                    for B_w in 1:length(all_locations[w].B_T)
                        bound_time = all_locations[u].B_T[B_u].min_time + distance_3(all_locations[u].id,all_locations[w].id)
                        if all_locations[w].B_T[B_w].max_time >= bound_time
                            push!(all_locations[u].B_T[B_u].forward, all_locations[w].B_T[B_w].id)
                            push!(all_locations[w].B_T[B_w].backward, all_locations[u].B_T[B_u].id)
                            break
                        end
                    end
                end
            end
        end
        ##############################################################################################
        ## connect the last and the first demand node from each customer u to the depot demand node.
        push!(all_locations[u].B_T[end].forward, length(all_time_nodes))
        push!(all_locations[u].B_T[1].backward, length(all_time_nodes))
        push!(all_time_nodes[end].forward,all_locations[u].B_T[1].id)
        push!(all_time_nodes[end].backward, all_locations[u].B_T[end].id )
        ##############################################################################################
    end
    return all_time_nodes
end

update_time_edges (generic function with 1 method)

In [28]:
function update_time_edges(sol::Solution)
    # Clear relevant lists before starting
    
    sol.time_nodes = []
    for u in 1:num_locations
        sol.customers[u].B_T = []
    end

    for u in 1:(num_locations-1)
        t_plus = sol.customers[u].W_T[1]
        id = length(sol.time_nodes) + 1
        push!(sol.time_nodes, Time_node(id, u, 1, sol.customers[u].time_window[1], t_plus, [], []))
        push!(sol.customers[u].B_T, sol.time_nodes[end])
        for t in 2:length(sol.customers[u].W_T)
            id = length(sol.time_nodes) + 1
            t_plus = sol.customers[u].W_T[t]
            t_minus = sol.customers[u].W_T[t-1] + 1
            push!(sol.time_nodes, Time_node(id, u, t, t_minus, t_plus, [], []))
            push!(sol.customers[u].B_T, sol.time_nodes[end])
        end
    end

    push!(sol.time_nodes, Time_node(length(sol.time_nodes) + 1, num_locations, 0, 0, sol.customers[end].time_window[2], [], []))
    push!(sol.customers[end].B_T, sol.time_nodes[end]) ## add the depot node to the depot. 
    for u in 1:num_locations-1
        for B_u in 1:length(sol.customers[u].B_T) - 1
            push!(sol.customers[u].B_T[B_u].forward, sol.customers[u].B_T[B_u+1].id)
            push!(sol.customers[u].B_T[B_u+1].backward, sol.customers[u].B_T[B_u].id)
        end
    end
            
    for u in 1:num_locations-1
        for B_u in 1:length(sol.customers[u].B_T)
            for w in 1:num_locations -1
                if u != w
                    for B_w in 1:length(sol.customers[w].B_T)
                        bound_time = sol.customers[u].B_T[B_u].min_time + distance_3(sol.customers[u].id, sol.customers[w].id)
                        if sol.customers[w].B_T[B_w].max_time >= bound_time
                            push!(sol.customers[u].B_T[B_u].forward, sol.customers[w].B_T[B_w].id)
                            push!(sol.customers[w].B_T[B_w].backward, sol.customers[u].B_T[B_u].id)
                            break
                        end
                    end
                end
            end
        end
        push!(sol.customers[u].B_T[end].forward, length(sol.time_nodes))
        push!(sol.customers[u].B_T[1].backward, length(sol.time_nodes))
        push!(sol.time_nodes[end].forward, sol.customers[u].B_T[1].id)
        push!(sol.time_nodes[end].backward, sol.customers[u].B_T[end].id)
    end
    return sol.time_nodes
end


update_time_edges (generic function with 2 methods)

In [29]:
all_demand_nodes = update_demand_edges()
all_time_nodes = update_time_edges()


101-element Vector{Any}:
 Time_node(1, 1, 1, 161.0f0, 161.0f0, [2], [101, 3, 4, 5, 6, 9, 10, 11, 12, 13  …  87, 88, 89, 90, 93, 94, 97, 98, 99, 100])
 Time_node(2, 1, 2, 162.0f0, 171.0f0, [101], [1, 51, 52, 91, 92])
 Time_node(3, 2, 1, 50.0f0, 50.0f0, [4, 1, 5, 7, 11, 16, 19, 25, 33, 36  …  67, 69, 73, 76, 79, 81, 85, 91, 95, 99], [101])
 Time_node(4, 2, 2, 51.0f0, 60.0f0, [1, 5, 7, 11, 16, 19, 25, 33, 36, 39  …  67, 69, 73, 79, 81, 85, 91, 95, 99, 101], [3, 83, 84])
 Time_node(5, 3, 1, 116.0f0, 116.0f0, [6, 1, 8, 25, 47, 49, 70, 96], [3, 4, 101, 9, 10, 23, 24, 27, 28, 41  …  71, 72, 77, 78, 83, 84, 89, 90, 93, 94])
 Time_node(6, 3, 2, 117.0f0, 126.0f0, [1, 8, 25, 47, 49, 70, 96, 101], [5, 17, 18, 21, 22, 29, 30, 45, 46, 79, 80])
 Time_node(7, 4, 1, 149.0f0, 149.0f0, [8, 49], [3, 4, 101, 9, 10, 11, 12, 13, 14, 17  …  81, 82, 83, 84, 87, 88, 89, 90, 93, 94])
 Time_node(8, 4, 2, 150.0f0, 159.0f0, [49, 101], [7, 5, 6, 15, 16, 51, 52])
 Time_node(9, 5, 1, 34.0f0, 34.0f0, [10, 1, 5, 7, 11, 

In [30]:
### debugging stuff
for b in 1:length(all_time_nodes)
    for i in all_time_nodes[b].forward
        found = false
        for j in all_time_nodes[i].backward
            if j == b
                found = true
                break
            end
        end
        if !found
            print("Demand b connects to i but i does not know about it  b,i ( $b , $i )")
        end
    end
end         

for b in 1:length(all_demand_nodes)
    for i in all_demand_nodes[b].forward
        found = false
        for j in all_demand_nodes[i].backward
            if j == b
                found = true
                break
            end
        end
        if !found
            print("Demand b connects to i but i does not know about it  b,i ( $b , $i )")
        end
    end
end   
    

# Algorithm 1
### Dynamic programming algorithm
### Initiate LA routes Ru
### Create non pareto dominated paths for each $u \in N$

- Label is a struct that stores information about the current path.

In [31]:
mutable struct Label
    cost::Float32                        
    time_early::Float32
    time_late::Float32
    load::Float32
    customer::Int             # Current node ID
    previous::Union{Label, Nothing}  # Previous label (for backtracking)
end

- contains_cust is a function that checks wheather the customer is already visited in a label. 

In [32]:
function contains_cust(L::Label, cust::Int)
    ## returns true if the costomer was visited in label L
    L_p = L
    while L_p != nothing
        c = L_p.customer
        if c == cust
            return true
        end
        L_p = L_p.previous
    end
    return false
end

contains_cust (generic function with 1 method)

In [33]:
function contains_all(L1::Label,L2::Label)
    # Returns true if set of visited customer are the same in both labels. 
    L1_p = L1
    L2_p = L2
    # check whether l1 is in l2
    while L1_p != nothing
        if !contains_cust(L2, L1_p.customer)
            return false
        end
        L1_p = L1_p.previous
    end
    # check whether L2 is in L1
    while L2_p != nothing
        if !contains_cust(L1, L2_p.customer)
            return false
        end
        L2_p = L2_p.previous
    end
    return true
end 

contains_all (generic function with 1 method)

- dominates is a function that compares two Labels that start and end in the same place while visiting the same set of customers. 

## Resource extension functions. 
- Here we build a set of functions that keep track of the resources used while expanding the labels from one state to the next. 

In [34]:
function resource_cost(L1::Label, cust2::Int)
    cost = L1.cost
    cost += distance_2(L1.customer, cust2)
    return cost
end

resource_cost (generic function with 1 method)

In [35]:
function resource_time_early(L1::Label, cust2::Int)
    time = L1.time_early + distance_3(L1.customer,cust2)
    time = max(all_locations[cust2].time_window[1], time)
    return time
end

resource_time_early (generic function with 1 method)

In [36]:
function resource_time_latest(L1::Label, cust2::Int)
    time = L1.time_late + distance_3(L1.customer, cust2)
    time = min(all_locations[cust2].time_window[2], time)
    return time
end

resource_time_latest (generic function with 1 method)

In [37]:
function resource_load(L1::Label, cust2::Int)
    load = L1.load
    load += all_locations[cust2].demand
    return load
end

resource_load (generic function with 1 method)

In [38]:
function label_extension(L::Label, cust::Int)
    cost = resource_cost(L,cust)
    early = resource_time_early(L, cust)
    late = resource_time_latest(L, cust)
    load = resource_load(L, cust)
    new_label = Label(cost, early, late, load, cust, L)
    return new_label
end

label_extension (generic function with 1 method)

In [39]:
function feasible(L::Label, cust::Int)
    time1 = resource_time_early(L, cust) # early time window 
    time  = (time1 <= all_locations[cust].time_window[2]) #bool if early time is less than late time window
    load = resource_load(L, cust) # total load has to be less than the capacity of the vehicle.
    return time && load <= vehicle_capacity && !contains_cust(L, cust)
end
    

feasible (generic function with 1 method)

In [40]:
function dominates(L1::Label, L2::Label)::Bool
    resources = L1.cost <= L2.cost && L1.time_early <= L2.time_early && L1.time_late -L1.cost>= L2.time_late - L1.cost && L1.customer == L2.customer
    return resources && contains_all(L1,L2)
end

dominates (generic function with 1 method)

In [41]:
function dominates_2(L1::Label, L2::Label, cust2::Int)::Bool
    cost1 = resource_cost(L1, cust2)
    early1 = resource_time_early(L1, cust2)
    late1 = resource_time_latest(L1,cust2)
    #################################
    cost2 = resource_cost(L2, cust2)
    early2 = resource_time_early(L2,cust2)
    late2 = resource_time_latest(L2,cust2)
    ###################################
    resources = cost1 <= cost2 && early1 <= early2 && late1 -cost1 >= late2 - cost2
    ###################################
    if feasible(L1,cust2)&& !feasible(L2,cust2)
        return true
    end
    return resources && contains_all(L1,L2)
end

dominates_2 (generic function with 1 method)

- build_path is a function that takes a label and using backtracking retrives the list of customers used in the path and returns a list with the correct sequence of customer visits from the start of the path to the end. 

In [42]:
function build_path(L::Label, future::Vector{Int},load, time)
    L_p = L
    path = []
    while L_p != nothing
        push!(path, L_p.customer)
        L_p = L_p.previous
    end
    reverse!(path)
    id = length(all_paths)
    id +=1
    push!(all_paths, Path(id ,path, future, load, time))
    return all_paths[end]
end

build_path (generic function with 1 method)

- from a label, it returns the first cutomer in path. 

In [43]:
function first_customer(L::Label)
    L_p = L
    first_customer = L_p.customer
    while L_p != nothing
        first_customer = L_p.customer
        L_p = L_p.previous
    end
    return first_customer
end

first_customer (generic function with 1 method)

## DP Algorithm 2

In [44]:
function non_dominated_paths_u(customer::Int) 
    cost = distance_3(all_locations[end].id,customer)
    time = max(all_locations[customer].time_window[1], cost)
    time2 = all_locations[customer].time_window[2]
    load = all_locations[customer].demand
    LA_paths = Dict{Tuple{Int, Int}, Vector{Label}}()
    LA_paths[(0, customer)] = [Label(cost, time, time2, load, customer, nothing)]
    set_size = length(all_locations[customer].N)
    for i in 0:(set_size-1)
        for cust1 in all_locations[customer].N
            if i == 0
                cust1 = customer
            end
            #get!(LA_paths, (i, cust1), [])
            if !haskey(LA_paths, (i, cust1)) ## check if we have visited this customer.
                continue
            end
            for L in LA_paths[(i, cust1)]
                for cust2 in all_locations[customer].N
                    if feasible(L,cust2)
                        L1 = label_extension(L,cust2)
                        dominated = false
                        LA_paths[(i + 1, cust2)] = get!(LA_paths, (i + 1, cust2), [])  # Create an empty list if it doesn't exist
                        for idx in length(LA_paths[(i + 1, cust2)]):-1:1
                            L2 = LA_paths[(i + 1, cust2)][idx]
                            if dominates(L2,L1)
                                dominated = true
                                break
                            end
                            if dominates(L1,L2)
                                deleteat!(LA_paths[(i + 1, cust2)], idx)
                            end
                        end
                        if !dominated
                            push!(LA_paths[(i + 1, cust2)], L1)
                        end 
                    end
                end
            end
            if i == 0
                break
            end
        end
    end

    #################################################################################################
    ############################ EXTEND TO ALL OTHER CUSTOMERS ######################################
    ############################ AND REMOVE THEM AFTERWARDS :) ######################################
    #################################################################################################
    
    LA_arcs = Dict{Int, Vector{Label}}()
    ## add the initial customer to the list, since it will not get dominated.
    LA_arcs[0] = LA_paths[(0, customer)]
    non_feasible = 0
    for i in 1:set_size
        for cust1 in all_locations[customer].N
            if !haskey(LA_paths, (i, cust1)) ## check if we have visited this customer.
                continue
            end
            for L in LA_paths[(i, cust1)] # this is the label that we want to add, potencially.
                dominated = true
                compare = false # did we compare something?
                get!(LA_arcs,i, [])
                if length(LA_arcs[i]) == 0
                    compare = true
                    dominated = false
                end
                for idx in length(LA_arcs[i]):-1:1
                    
                    L2 = LA_arcs[i][idx] # this is the label that we want to use in dominance
                    dominated_2 = true
                    
                    if !contains_all(L,L2) ## if these labels do not represent the same set of customers continue
                        continue
                    end 
                    
                    for cust2 in 1:length(all_locations) ## for all customers in the insance not in N
                        if cust2 in all_locations[customer].N || cust2 == customer
                            continue
                        end
                        if !feasible(L,cust2)
                            continue
                        end
                        compare = true
                        ############
                        ## check which label dominates for this customer
                        if dominates_2(L,L2,cust2)
                            dominated = false
                        elseif dominates_2(L2,L,cust2)
                            dominanted_2 = false
                        else
                            dominated_2 = false
                            dominated = false
                            break
                        end
                    end
                    if dominated_2
                        deleteat!(LA_arcs[i],idx)
                    end
                    if compare && dominated
                        break
                    end
                end
                if compare && dominated
                    continue
                else
                    push!(LA_arcs[i], L)
                end
            end
        end
    end

    paths = Vector{Path}()
    
    for (key, value) in LA_paths
        
        for L in value 
            
            #first = first_customer(L)
            future = Vector{Int}()
            #for kust in all_locations
            #    if !(kust.id in all_locations[first].N)
            #        if feasible(L, kust.id)
            #            push!(future, kust.id)
            #        end
            #    end
            #end
            
            push!(paths, build_path(L, future, L.load, L.time_early))
        end
    end
    return paths
end

non_dominated_paths_u (generic function with 1 method)

In [45]:
#non_dominated_paths_u(10)

#### for every customer we build all LA arcs using previous funcitons

In [46]:
total_time = 0
total_memory = 0
empty!(all_paths)
for i in all_locations[1:end-1]
    i.paths, time, allocations, memory = @timed non_dominated_paths_u(i.id) 
    total_time+=time
    total_memory += memory
end

println("Time: ", total_time, " seconds")
println("Memory: ", total_memory, " bytes")
all_paths

Time: 0.06435058299999998 seconds
Memory: 0.0 bytes


778-element Vector{Path}:
 Path(1, [1], Int64[], 10.0f0, 161.0f0)
 Path(2, [2], Int64[], 7.0f0, 50.0f0)
 Path(3, [2, 21], Int64[], 18.0f0, 70.4f0)
 Path(4, [2, 21, 40, 37], Int64[], 35.0f0, 134.0f0)
 Path(5, [2, 21, 22, 37], Int64[], 44.0f0, 134.0f0)
 Path(6, [2, 21, 41, 37], Int64[], 31.0f0, 134.0f0)
 Path(7, [2, 13], Int64[], 30.0f0, 159.0f0)
 Path(8, [2, 21, 41], Int64[], 23.0f0, 97.0f0)
 Path(9, [2, 40], Int64[], 16.0f0, 85.0f0)
 Path(10, [2, 21, 22], Int64[], 36.0f0, 97.0f0)
 Path(11, [2, 21, 40, 13], Int64[], 50.0f0, 159.0f0)
 Path(12, [2, 21, 22, 13], Int64[], 59.0f0, 159.0f0)
 Path(13, [2, 21, 41, 13], Int64[], 46.0f0, 159.0f0)
 ⋮
 Path(767, [49, 1], Int64[], 40.0f0, 161.0f0)
 Path(768, [49, 17], Int64[], 32.0f0, 157.0f0)
 Path(769, [49, 13], Int64[], 53.0f0, 167.2f0)
 Path(770, [49], Int64[], 30.0f0, 108.0f0)
 Path(771, [49, 32], Int64[], 53.0f0, 147.0f0)
 Path(772, [50, 48], Int64[], 49.0f0, 168.3f0)
 Path(773, [50, 25], Int64[], 19.0f0, 172.0f0)
 Path(774, [50, 24], Int64[],

- check_edge is a function given a list, and two customers as input, it checks that the edge of those two customers exists 

In [47]:
for u in all_locations
    if u.time_window[1] < distance_3(all_locations[end].id, u.id)
        println(" time window is bellow reality ", u.id, " ", distance_3(all_locations[end].id, u.id) , " ", u.time_window[1])
    end
end
println(all_locations[36].W_T)

[42.0, 51.0]


In [48]:
function set_contains_all(p1::Vector{Int}, p2::Vector{Int})
    if length(p1) > length(p2)
        return false
    end
    for p in p1
        contains = false
        for t in p2
            if p == t
                contains = true
                break
            end
        end
        if !contains
            return false
        end
    end
    return true
end

set_contains_all (generic function with 1 method)

In [49]:
function check_edge(w::Int, v::Int, p::Vector{Int})::Bool
    # check to see if w is followd by v, i.e.,  a_w,v,r
    for i in 1:(length(p)-1)
        if p[i] == w && p[i+1] == v
            return true
        end
    end
    return false
end

check_edge (generic function with 1 method)

In [50]:
function split_until_v(vec::Vector{Int}, u::Int)
    # Find the first occurrence of u
    index = findfirst(==(u), vec)
    
    if index !== nothing
        if index < length(vec)
        # Return the sublist from the first element until `u`
            return vec[1:index], vec[index+1]
        else
            return vec, u
        end
    else
        # Return the entire list if `u` is not found
        return vec , vec[end]
    end
end


split_until_v (generic function with 1 method)

#### Optimization Model
- is a struct that stores information about the model that we solve.
- dual variables, constraints and the model. 

In [51]:

mutable struct OptimizationModel
    model::JuMP.Model               
    objective::Float64
    vehicle::Any
    load::Dict{Int,Any}
    time::Dict{Int,Any}
    x::Dict{Tuple{Int, Int}, Any}
    y::Dict{Int, Any}
    z_d::Dict{Tuple{Int, Int}, Any}
    z_t::Dict{Tuple{Int, Int}, Any}
    c8a::Dict{Tuple{Int,Int,Int,Int}, Any}
    c8b::Dict{Tuple{Int,Int,Int}, Any}
    c6h::Dict{Tuple{Int,Int,Int}, Any}       
    c6i::Dict{Tuple{Int,Int}, Any}
    c6j::Dict{Int, Any}
    c6k::Dict{Tuple{Int,Int}, Any}
    c6l::Dict{Int, Any}
    c6m::Dict{Tuple{Int,Int}, Any}
    pi_8a::Dict{Tuple{Int,Int,Int,Int}, Float64}
    pi_8b::Dict{Tuple{Int,Int,Int}, Float64}
    pi_6j::Dict{Int, Float64}
    pi_6l::Dict{Int, Float64}
end
    

### Creates the linear relaxation problem 

In [52]:
function create_model!(larcs::Bool, sol::Solution)
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    #model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    # Dictionaries for variables and constraints
    load = Dict{Int,JuMP.VariableRef}()
    arrival_time = Dict{Int,JuMP.VariableRef}()
    x_var = Dict{Tuple{Int, Int}, JuMP.VariableRef}()
    y_var = Dict{Int, JuMP.VariableRef}()
    z_D = Dict{Tuple{Int, Int}, JuMP.VariableRef}()
    z_T = Dict{Tuple{Int, Int}, JuMP.VariableRef}()
    c8a = Dict{Tuple{Int, Int, Int, Int}, ConstraintRef}()
    c8b = Dict{Tuple{Int, Int, Int}, ConstraintRef}()
    c6h =Dict{Tuple{Int, Int, Int}, ConstraintRef}()
    c6i =Dict{Tuple{Int, Int}, ConstraintRef}()
    c6j = Dict{Int, ConstraintRef}()
    c6k = Dict{Tuple{Int,Int}, ConstraintRef}()
    c6l = Dict{Int, ConstraintRef}()
    c6m = Dict{Tuple{Int,Int}, ConstraintRef}()

    
    # Minimum number of vehicle needed
    
    dsum = 0
    for i in 1:num_locations
        dsum += sol.customers[i].demand
    end
    min_vehicles = ceil(dsum/vehicle_capacity)
    # Variables

    for i in 1:num_locations, j in 1:num_locations
        x_var[(i, j)] = @variable(model, base_name="x_$(i)_$(j)", lower_bound=0, upper_bound=1)
    end
    
    @variable(model, vehicles >= min_vehicles)
    
    for i in 1:num_locations
        load[i]= @variable(model, base_name="load_$(i)", lower_bound= all_locations[i].demand , upper_bound = vehicle_capacity)
        arrival_time[i]= @variable(model, base_name="time_$(i)", lower_bound= all_locations[i].time_window[1] , upper_bound = all_locations[i].time_window[2])
    end
    

    # Objective: Minimize total distance
    @objective(model, Min, sum(distance_2(i, j) * x_var[(i, j)] for i in 1:num_locations, j in 1:num_locations if i != j))

    # Constraints
    # A route must leave a customer exactly once.

    for i in 1:(num_locations - 1)  # Exclude the depot
        @constraint(model, sum(x_var[(i, j)] for j in 1:num_locations if i != j) == 1)
    end

    # A route must arrive at a customer exactly once.

    for j in 1:(num_locations - 1)  # Exclude the depot
        @constraint(model, sum(x_var[(i, j)] for i in 1:num_locations if i != j) == 1)
    end

    
    # Capacity constraints
    
    for u in 1:(num_locations-1), v in 1:num_locations
        if v != u
            demand_u = sol.customers[u].demand
            @constraint(model, load[u] >= load[v] + demand_u - (vehicle_capacity + demand_u)*(1 - x_var[(v, u)]))
        end
    end
    # Respect time windows
    for i in 1:(num_locations - 1), j in 1:num_locations
        if i != j
            upper_time_i = sol.customers[i].time_window[2]
            dist_ij = distance_3(i, j)
            @constraint(model, arrival_time[j] >= arrival_time[i] + dist_ij - (dist_ij + upper_time_i)*(1 - x_var[(i, j)]))
        end
    end
   
    # bounds
    for i in 1:num_locations
        minim = distance_3(sol.customers[end].id, i)
        minim = ceil(minim)
        set_lower_bound(arrival_time[i], max(sol.customers[i].time_window[1], minim))  # Earliest arrival
        set_upper_bound( arrival_time[i], sol.customers[i].time_window[2]) # Latest arrival
    end

    @constraint(model, sum(x_var[(num_locations, i)] for i in 1:(num_locations-1)) == vehicles)

    ####################################################################################################
    ##############################      NEW COMPACT FORMULATION       ##################################
    ####################################################################################################
   
    
 
    for p in sol.paths
        i =p.id
        y_var[i] = @variable(model, base_name="y_$(i)", lower_bound=0, upper_bound=1)
    end

    
    ####################################################################################################
    ###########  If not MILP we do the following constraints. with constraints 8a, 8b. #################
    ####################################################################################################

    # LA constraints

    for i in 1:(num_locations-1)
        pp = sol.customers[i].paths
        @constraint(model, sum(y_var[r.id] for r in pp) == 1)
    end

    for i in 1:(num_locations-1)
        sort!(sol.customers[i].paths, by = element -> length(element.path))
    end
     
    for u in 1:(num_locations-1)
        for k in 1:length(sol.customers[u].N) # for every size of neighborhood
            k_neigh = sol.customers[u].N[1:k] # this is the new neighborhood
            k_neigh_u = [u]
            append!(k_neigh_u, k_neigh)
                
            for w in k_neigh_u
                for v in k_neigh
                    if w == v
                        continue
                    end
                    index = []
                    for r in sol.customers[u].paths 
                        partial, gg = split_until_v(r.path, v)
                        if set_contains_all(partial, k_neigh_u) && check_edge(w,v,partial)
                            push!(index,r.id)
                        end
                    end
                    if !isempty(index)
                        c8a[(u,w,v,k)] = @constraint(model,0.0001*k + x_var[(w,v)] >= sum(y_var[id] for id in index))#c8a
                    end
                end
            end
        end
    end
    ####################################################################################################

    for u in 1:(num_locations-1)
        for k in 1:length(sol.customers[u].N)
            k_neigh = sol.customers[u].N[1:k] # this is the new neighborhood
            k_neigh_u = [u]
                
            append!(k_neigh_u, k_neigh)
            k_complement = [j for j in 1:num_locations if !(j in k_neigh_u)]
            for w in k_neigh_u 
                index = []
                for r in sol.customers[u].paths
                    if w == r.path[end]  && set_contains_all(r.path, k_neigh_u)
                        push!(index,r.id)
                    else
                        partial, v = split_until_v(r.path, w)
                        if set_contains_all(partial,k_neigh_u) && !(v in k_neigh_u)
                            #println(" neig_ku>> ",k_neigh_u, " path >>> " ,r.path, " partial>>  " , partial)
                            push!(index,r.id) #partial, v = split_until_v(r.path, w)
                        end
                    end
                end
                if !isempty(index)
                    c8b[(u,w,k)] = @constraint(model, 0.0001*k + sum(x_var[(w,v)] for v in k_complement) >= sum(y_var[rr] for rr in index)) #c8b
                end
            end
        end
    end
    # Demand constraints
    ####################################################################################################
    # Create an empty dictionary to store variables

     
    for node in sol.demand_nodes
        i = node.id
        for j in node.forward
            z_D[(i, j)] = @variable(model, base_name="z_d$(i)_$(j)", lower_bound=0, upper_bound=1)
        end
    end

    for i in 1:length(sol.demand_nodes) ## no depot here 
        c6j[i] = @constraint(model, sum(z_D[(i,j)] for j in sol.demand_nodes[i].forward) == 
            sum(z_D[(j,i)] for j in sol.demand_nodes[i].backward)) # c6j
    end

    for u in 1:(num_locations)
        for v in 1:(num_locations)
            if u == v
                continue
            end
            c6k[(u,v)] = @constraint(model, x_var[(u, v)] == 
                sum(z_D[(i, j)] for node1 in sol.customers[u].B_D, j in node1.forward, i = node1.id if sol.demand_nodes[j].customer == v))
        end
    end


    ####################################################################################################

    ## Flow constraints for all nodes in the time graph . 

    for node in sol.time_nodes
        i = node.id
        for j in node.forward
            z_T[(i, j)] = @variable(model, base_name="z_t$(i)_$(j)", lower_bound=0, upper_bound=1)
        end
    end
    
    # Flow conservation for time nodes
    for i in 1:length(sol.time_nodes) ## no depot here 
        c6l[i] = @constraint(model, sum(z_T[(i, j)] for j in sol.time_nodes[i].forward ) == 
               sum(z_T[(j, i)] for j in sol.time_nodes[i].backward ) ) # c6l
    end

        # Time consistency constraints
    for u in 1:num_locations
        for v in 1:num_locations
            if u == v
                continue
            end
            c6m[(u,v)]= @constraint(model, x_var[(u, v)] == 
                sum(z_T[(i, j)] for node1 in sol.customers[u].B_T, j in node1.forward, i = node1.id if sol.time_nodes[j].customer == v))
        end
    end 

    ##########################################################################################################

    return OptimizationModel(model, 0, vehicles,load, arrival_time, x_var,y_var, z_D,z_T,c8a,c8b,c6h,c6i, c6j, c6k, c6l, c6m,Dict(),Dict(),Dict(),Dict())
            #return x, y, z_D, z_T, constraints, constraints_2, model
end

create_model! (generic function with 1 method)

### Creates the Milp for the final solution

In [53]:
function create_MILP!(sol::Solution)
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    #model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 1)
    # Dictionaries for variables and constraints
    load = Dict{Int,JuMP.VariableRef}()
    arrival_time = Dict{Int,JuMP.VariableRef}()
    x_var = Dict{Tuple{Int, Int}, JuMP.VariableRef}()
    y_var = Dict{Int, JuMP.VariableRef}()
    z_D = Dict{Tuple{Int, Int}, JuMP.VariableRef}()
    z_T = Dict{Tuple{Int, Int}, JuMP.VariableRef}()
    c8a = Dict{Tuple{Int, Int, Int, Int}, ConstraintRef}()
    c8b = Dict{Tuple{Int, Int, Int}, ConstraintRef}()
    c6h =Dict{Tuple{Int, Int, Int}, ConstraintRef}()
    c6i =Dict{Tuple{Int, Int}, ConstraintRef}()
    c6j = Dict{Int, ConstraintRef}()
    c6k = Dict{Tuple{Int,Int}, ConstraintRef}()
    c6l = Dict{Int, ConstraintRef}()
    c6m = Dict{Tuple{Int,Int}, ConstraintRef}()

    
    # Minimum number of vehicle needed
    
    dsum = 0
    for i in 1:num_locations
        dsum += sol.customers[i].demand
    end
    min_vehicles = ceil(dsum/vehicle_capacity)
    # Variables
   
    @variable(model, x_var[i=1:num_locations, j=1:num_locations; i != j], Bin) # Binary decision: 1 if travel from i to j
    @variable(model, vehicles >= min_vehicles, Int)
    MOI.set(model, Gurobi.VariableAttribute("BranchPriority"), vehicles, 10)
    
    for i in 1:num_locations
        load[i]= @variable(model, base_name="load_$(i)", lower_bound= all_locations[i].demand , upper_bound = vehicle_capacity)
        arrival_time[i]= @variable(model, base_name="time_$(i)", lower_bound= all_locations[i].time_window[1] , upper_bound = all_locations[i].time_window[2])
    end
    

    # Objective: Minimize total distance
    @objective(model, Min, sum(distance_2(i, j) * x_var[(i, j)] for i in 1:num_locations, j in 1:num_locations if i != j))

    # Constraints
    # A route must leave a customer exactly once.

    for i in 1:(num_locations - 1)  # Exclude the depot
        @constraint(model, sum(x_var[(i, j)] for j in 1:num_locations if i != j) == 1)
    end

    # A route must arrive at a customer exactly once.

    for j in 1:(num_locations - 1)  # Exclude the depot
        @constraint(model, sum(x_var[(i, j)] for i in 1:num_locations if i != j) == 1)
    end

    
    # Capacity constraints
    
    for u in 1:(num_locations-1), v in 1:num_locations
        if v != u
            demand_u = sol.customers[u].demand
            @constraint(model, load[u] >= load[v] + demand_u - (vehicle_capacity + demand_u)*(1 - x_var[(v, u)]))
        end
    end
    # Respect time windows
    for i in 1:(num_locations - 1), j in 1:num_locations
        if i != j
            upper_time_i = sol.customers[i].time_window[2]
            dist_ij = distance_3(i, j)
            @constraint(model, arrival_time[j] >= arrival_time[i] + dist_ij - (dist_ij + upper_time_i)*(1 - x_var[(i, j)]))
        end
    end
   
    # bounds
    for i in 1:num_locations
        minim = distance_3(sol.customers[end].id, i)
        minim = ceil(minim)
        set_lower_bound(arrival_time[i], max(sol.customers[i].time_window[1], minim))  # Earliest arrival
        set_upper_bound( arrival_time[i], sol.customers[i].time_window[2]) # Latest arrival
    end

    @constraint(model, sum(x_var[(num_locations, i)] for i in 1:(num_locations-1)) == vehicles)

    ####################################################################################################
    ##############################      NEW COMPACT FORMULATION       ##################################
    ####################################################################################################
    set_of_variables = []

    for u in 1:(num_locations - 1)
        temp_set = union(sol.customers[u].N, [u])
        for w in temp_set
            for v in sol.customers[u].N
                if v != w
                    for pp in sol.customers[u].paths
                        if check_edge(w,v, pp.path)
                            push!(set_of_variables, pp.id)
                            
                        end
                    end
                end
            end
        end
    end

    for u in 1:(num_locations - 1)
        temp_set = union(sol.customers[u].N, [u])
        for w in temp_set
            for pp in sol.customers[u].paths
                if pp.path[end] == w
                    push!(set_of_variables, pp.id)
                end
            end
        end
    end

    set_of_variables = unique(set_of_variables)
    
    for i in set_of_variables
        y_var[i] = @variable(model, base_name="y_$(i)", lower_bound=0, upper_bound=1)
    end

    
    ####################################################################################################
    ###########  If not MILP we do the following constraints. with constraints 8a, 8b. #################
    ####################################################################################################

        # LA constraints

    for i in 1:(num_locations-1)
        @constraint(model, sum(y_var[r] for r in set_of_variables if sol.paths[r].path[1] == i ) == 1)
    end

    for u in 1:(num_locations - 1)
        temp_set = union(sol.customers[u].N, [u])
        for w in temp_set
            for v in sol.customers[u].N
                if v != w
                   c6h[(u,w,v)]  = @constraint(model, x_var[(w, v)] >= sum(y_var[pp.id] for pp in sol.customers[u].paths if check_edge(w,v, pp.path)))
                end
            end
        end
    end
             
    for u in 1:(num_locations - 1)
        temp_set = union(sol.customers[u].N, [u])
        for w in temp_set
            c6i[(u,w)] = @constraint(model, sum(x_var[(w, v)] for v in 1:num_locations if !(v in temp_set)) 
                >= sum(y_var[pp.id] for pp in sol.customers[u].paths if pp.path[end] == w))
        end
    end

    # Demand constraints
    ####################################################################################################
    # Create an empty dictionary to store variables

     
    for node in sol.demand_nodes
        i = node.id
        for j in node.forward
            z_D[(i, j)] = @variable(model, base_name="z_d$(i)_$(j)", lower_bound=0, upper_bound=1)
        end
    end

    for i in 1:length(sol.demand_nodes) ## no depot here 
        c6j[i] = @constraint(model, sum(z_D[(i,j)] for j in sol.demand_nodes[i].forward) == 
            sum(z_D[(j,i)] for j in sol.demand_nodes[i].backward)) # c6j
    end

    for u in 1:(num_locations)
        for v in 1:(num_locations)
            if u == v
                continue
            end
            c6k[(u,v)] = @constraint(model, x_var[(u, v)] == 
                sum(z_D[(i, j)] for node1 in sol.customers[u].B_D, j in node1.forward, i = node1.id if sol.demand_nodes[j].customer == v))
        end
    end


        ####################################################################################################

    ## Flow constraints for all nodes in the time graph . 

    for node in sol.time_nodes
        i = node.id
        for j in node.forward
            z_T[(i, j)] = @variable(model, base_name="z_t$(i)_$(j)", lower_bound=0, upper_bound=1)
        end
    end
    
        # Flow conservation for time nodes
    for i in 1:length(sol.time_nodes) ## no depot here 
        c6l[i] = @constraint(model, sum(z_T[(i, j)] for j in sol.time_nodes[i].forward ) == 
            sum(z_T[(j, i)] for j in sol.time_nodes[i].backward ) ) # c6l
    end

    # Time consistency constraints
    for u in 1:num_locations
        for v in 1:num_locations
            if u == v
                continue
            end
            c6m[(u,v)]= @constraint(model, x_var[(u, v)] == 
                sum(z_T[(i, j)] for node1 in sol.customers[u].B_T, j in node1.forward, i = node1.id if sol.time_nodes[j].customer == v))
        end
    end 
    ##########################################################################################################
        
    return model
end

create_MILP! (generic function with 1 method)

## Model MILP 

In [54]:
# debugging stuff
#sol = Solution(all_demand_nodes, all_time_nodes, all_locations, all_paths)
#model_01 = create_model!(false, sol)
#solve!(model_01)
#model_01.objective

In [55]:
#for i in 1:num_locations
#    println(all_locations[i].W_T)
#end

### function that updates the demand flow variables and constraints in the model 

In [56]:
function update_model_demand!(lp::OptimizationModel, sol_1::Solution, sol::Solution)
    #################################################################
    ##########  create node variables if they do not exist   ########
    #################################################################
    # Clear variables and constraints from previous solutions.
    for u in 1:num_locations
        for node in sol_1.customers[u].B_D  # B_T -> B_D
            i = node.id
            for j in node.forward
                v = sol_1.demand_nodes[j].customer  # time_nodes -> demand_nodes
                JuMP.set_normalized_coefficient(lp.c6j[i], lp.z_d[(i,j)], 0)  # c6l -> c6j, z_t -> z_d
                JuMP.set_normalized_coefficient(lp.c6j[j], lp.z_d[(i,j)], 0)  # c6l -> c6j, z_t -> z_d
                if u != v
                    JuMP.set_normalized_coefficient(lp.c6k[(u,v)], lp.z_d[(i,j)], 0)  # c6m -> c6k, z_t -> z_d
                end
            end
        end
    end

    # Create constraints.
    for u in 1:num_locations
        for node in sol.customers[u].B_D  # B_T -> B_D
            i = node.id
            for j in node.forward
                if !haskey(lp.z_d, (i,j))  # z_t -> z_d
                    lp.z_d[(i, j)] = @variable(lp.model, base_name="z_d$(i)_$(j)", lower_bound=0, upper_bound=1)
                end
                if !haskey(lp.c6j, i)  # c6l -> c6j
                    lp.c6j[i] = @constraint(lp.model, 0 == 0)
                end
                if !haskey(lp.c6j, j)  # c6l -> c6j
                    lp.c6j[j] = @constraint(lp.model, 0 == 0)
                end
                v = sol.demand_nodes[j].customer  # time_nodes -> demand_nodes
                JuMP.set_normalized_coefficient(lp.c6j[i], lp.z_d[(i,j)], 1)  # c6l -> c6j, z_t -> z_d
                JuMP.set_normalized_coefficient(lp.c6j[j], lp.z_d[(i,j)], -1)  # c6l -> c6j, z_t -> z_d
                if u != v
                    JuMP.set_normalized_coefficient(lp.c6k[(u,v)], lp.z_d[(i,j)], -1)  # c6m -> c6k, z_t -> z_d
                end
            end
        end
    end
end

update_model_demand! (generic function with 1 method)

In [57]:
function update_model_time!(lp::OptimizationModel, sol_1::Solution, sol::Solution)

    #################################################################
    ##########  create node variables if they do not exist   ########
    #################################################################
    #clear variables and constraints from previous solutions. 
    for u in 1:num_locations
        for node in sol_1.customers[u].B_T
            i = node.id
            for j in node.forward
                v = sol_1.time_nodes[j].customer
                JuMP.set_normalized_coefficient(lp.c6l[i], lp.z_t[(i,j)], 0)
                JuMP.set_normalized_coefficient(lp.c6l[j], lp.z_t[(i,j)], 0)
                if u != v
                    JuMP.set_normalized_coefficient(lp.c6m[(u,v)], lp.z_t[(i,j)], 0) ## Eliminate all previous variables from these constraints.
                end
            end
        end
    end

    # create constraints. 
    for u in 1:num_locations
        for node in sol.customers[u].B_T
            i = node.id
            for j in node.forward
                if !haskey(lp.z_t, (i,j))
                    lp.z_t[(i, j)] = @variable(lp.model, base_name="z_t$(i)_$(j)", lower_bound=0, upper_bound=1)
                end
                if !haskey(lp.c6l,i)
                    lp.c6l[i] = @constraint(lp.model, 0==0) # 
                end
                if !haskey(lp.c6l,j)
                    lp.c6l[j] = @constraint(lp.model, 0==0) # 
                end
                v = sol.time_nodes[j].customer
                JuMP.set_normalized_coefficient(lp.c6l[i], lp.z_t[(i,j)], 1)
                JuMP.set_normalized_coefficient(lp.c6l[j], lp.z_t[(i,j)], -1)
                if u != v
                    JuMP.set_normalized_coefficient(lp.c6m[(u,v)], lp.z_t[(i,j)], -1)
                end
            end
        end
    end
end

update_model_time! (generic function with 1 method)

## Function that updates the la arcs in the model

In [58]:
function update_model_la!(lp::OptimizationModel, sol_1::Solution, sol::Solution)
    ## sol_1 is the previous solution
    ####################################################################################################
    ###################################  constraints 8a   ##############################################
    ####################################################################################################
    for u in 1:(num_locations-1)
        if length(sol.customers[u].N) < length(sol_1.customers[u].N)
            ####################### contracting sets #################################################
            for k in (length(sol.customers[u].N)+1):length(sol_1.customers[u].N) # for every size of neighborhood
                k_neigh = sol_1.customers[u].N[1:k] # this is the new neighborhood
                k_neigh_u = [u]
                append!(k_neigh_u, k_neigh)
                
                for w in k_neigh_u
                    for v in k_neigh
                        if w == v
                            continue
                        end
                        index = []
                        for r in sol_1.customers[u].paths 
                            partial, gg = split_until_v(r.path, v)
                            if set_contains_all(partial, k_neigh_u) && check_edge(w,v,partial)
                                push!(index,r.id)
                            end
                        end
                        if !isempty(index)
                            JuMP.set_normalized_coefficient(lp.c8a[u,w,v,k], lp.x[(w,v)], 0)
                            for i in index
                                JuMP.set_normalized_coefficient(lp.c8a[u,w,v,k], lp.y[i], 0)
                            end
                        end
                    end
                end
            end
        elseif length(sol_1.customers[u].N) < length(sol.customers[u].N)
            ####################### contracting sets #################################################
            for k in (length(sol_1.customers[u].N)+1):length(sol.customers[u].N) # for every size of neighborhood
                k_neigh = sol.customers[u].N[1:k] # this is the new neighborhood
                k_neigh_u = [u]
                append!(k_neigh_u, k_neigh)
                for w in k_neigh_u
                    for v in k_neigh
                        if w == v
                            continue
                        end
                        index = []
                        for r in sol.customers[u].paths 
                            partial, gg = split_until_v(r.path, v)
                            if set_contains_all(partial, k_neigh_u) && check_edge(w,v,partial)
                                push!(index,r.id)
                            end
                        end
                        if !isempty(index)
                            if !haskey(c8a,(u,w,v,k))
                                c8a[(u,w,v,k)] = @constraint(model,0.0001*k + x[(w,v)] >= sum(y[id] for id in index))#c8a
                            else
                                JuMP.set_normalized_coefficient(lp.c8a[u,w,v,k], lp.x[(w,v)], 1)
                                for i in index
                                    JuMP.set_normalized_coefficient(lp.c8a[u,w,v,k], lp.y[i], - 1)
                                end
                            end
                        end
                    end
                end
            end
        else
            continue
        end
    end
    ####################################################################################################
    ####################################################################################################
    ###################################  constraints 8b   ##############################################
    ####################################################################################################

    for u in 1:(num_locations-1)
        if length(sol.customers[u].N) < length(sol_1.customers[u].N)
        ####################### contracting sets #################################################
            for k in (length(sol.customers[u].N)+1):length(sol_1.customers[u].N)
                k_neigh = sol_1.customers[u].N[1:k] # this is the new neighborhood
                k_neigh_u = [u]
                
                append!(k_neigh_u, k_neigh)
                k_complement = [j for j in 1:num_locations if !(j in k_neigh_u)]
                for w in k_neigh_u 
                    index = []
                    for r in sol_1.customers[u].paths
                        if w == r.path[end]  && set_contains_all(r.path, k_neigh_u)
                            push!(index,r.id)
                        else
                            partial, v = split_until_v(r.path, w)
                            if set_contains_all(partial,k_neigh_u) && !(v in k_neigh_u)
                                #println(" neig_ku>> ",k_neigh_u, " path >>> " ,r.path, " partial>>  " , partial)
                                push!(index,r.id) #partial, v = split_until_v(r.path, w)
                            end
                        end
                    end
                    if !isempty(index)
                        for v in k_complement
                            JuMP.set_normalized_coefficient(lp.c8b[u,w,k], lp.x[(w,v)], 0)
                        end
                        for i in index
                            JuMP.set_normalized_coefficient(lp.c8b[u,w,k], lp.y[i], 0)
                        end
                    end
                end
            end
        elseif length(sol_1.customers[u].N) < length(sol.customers[u].N)
            ####################### increasing sets #################################################
            for k in (length(sol_1.customers[u].N)+1):length(sol.customers[u].N)
                k_neigh = sol.customers[u].N[1:k] # this is the new neighborhood
                k_neigh_u = [u]
                
                append!(k_neigh_u, k_neigh)
                k_complement = [j for j in 1:num_locations if !(j in k_neigh_u)]
                for w in k_neigh_u 
                    index = []
                    for r in sol.customers[u].paths
                        if w == r.path[end]  && set_contains_all(r.path, k_neigh_u)
                            push!(index,r.id)
                        else
                            partial, v = split_until_v(r.path, w)
                            if set_contains_all(partial,k_neigh_u) && !(v in k_neigh_u)
                                push!(index,r.id)
                            end
                        end
                    end
                    if !isempty(index)
                        if !haskey(c8b,(u,w,k))
                            c8b[(u,w,k)] = @constraint(model, 0.0001*k + sum(x[(w,v)] for v in k_complement) >= sum(y[rr] for rr in index)) #c8b
                        else
                            for v in k_complement
                               JuMP.set_normalized_coefficient(lp.c8b[u,w,k], x[(w,v)], 1)
                            end
                            for rr in index
                                JuMP.set_normalized_coefficient(lp.c8b[u,w,k], y[rr], -1)
                            end   
                        end
                    end
                end
            end
        else
            continue
        end
    end
end

update_model_la! (generic function with 1 method)

In [59]:
#=for (key, var) in model_01.time
    time = value(var)
    println("customer $key, time = $time")
end=#

In [60]:
#=for (key, var) in model_01.z_t
    val = value(var)
    if val > 0.0001
        node1 = sol.time_nodes[key[1]]
        node2= sol.time_nodes[key[2]]
        if node1.customer == 20 || node2.customer == 20
            println("customer $key, variable = $val, ", node1.customer," -->>", node2.customer)
        end
    end
end=#

In [61]:
#=for i in all_locations
    if i.id == 11
        for j in i.B_D
            println(i.id, " = [",j.min_load ,", ",j.max_load, "]")
        end
    end
end=#

### Function that converst a linear problem to the binary problem This is not necessary in this implementation. Since we create a model at the end. 

In [62]:
function convert_model_MILP!(MILP::OptimizationModel)
    for (key, var) in MILP.x
        JuMP.set_binary(var)  # Change to binary
    end
    ## set the total number of vehicle to be an interger variable with high branching priority
    JuMP.set_integer(MILP.vehicle)
    MOI.set(MILP.model, Gurobi.VariableAttribute("BranchPriority"), MILP.vehicle, 10)
    
    ## set the outputflag to 1 to visualize the results from gurobi
    set_optimizer_attribute(MILP.model, "OutputFlag", 1)
end

convert_model_MILP! (generic function with 1 method)

In [63]:
#convert_model_MILP!(model_01)

In [64]:
#optimize!(model_01.model)

### Algorithm 1

### Function that solves an object of Optimization Model

In [65]:
function solve!(MILP::OptimizationModel)
    # Solve the model
    optimize!(MILP.model)

    # Store the objective value
    if JuMP.termination_status(MILP.model) == MOI.OPTIMAL
        MILP.objective = JuMP.objective_value(MILP.model)
        #println("Objective value: ", MILP.objective)
    else
        println("Solution not optimal. Status: ", JuMP.termination_status(MILP.model))
        MILP.objective = NaN  # Indicate no valid objective value
    end

    # Store dual values for constraints
    for (key, constraint) in MILP.c8a
        MILP.pi_8a[key] = JuMP.dual(constraint)
    end

    for (key, constraint) in MILP.c8b
        MILP.pi_8b[key] = JuMP.dual(constraint)
    end

    for (key, constraint) in MILP.c6j
        MILP.pi_6j[key] = JuMP.dual(constraint)
    end

    for (key, constraint) in MILP.c6l
        MILP.pi_6l[key] = JuMP.dual(constraint)
    end
    return MILP  # Return updated object
end

solve! (generic function with 1 method)

In [66]:
function contract_neighbors!(MILP::OptimizationModel, sol::Solution)
    value = Dict()
    max_k_u = Dict()
    
    for u in 1:num_locations-1
        max_k_u[u] = 1
        for k in 1:length(sol.customers[u].N)
            value[(u,k)] = 0
        end
    end

    for (key, dual) in MILP.pi_8a
        key_u, key_w, key_v, key_k = key
        if haskey(value, (key_u,key_k))
            value[(key_u,key_k)] += dual
        end
    end
   
    for (key, dual) in MILP.pi_8b
        key_u, key_w, key_k = key
        if haskey(value, (key_u,key_k))
            value[(key_u,key_k)] += dual
        end
    end

    for u in 1:num_locations-1
        max_k = 0
        for k in 1:length(sol.customers[u].N)
            if value[(u,k)]> 0
                if max_k < k
                    max_k = k
                end
            end
        end
        max_k_u[u] = max_k
    end

    ## Now we contract the set for all customers
    for u in 1:num_locations-1
        if max_k_u[u] > 0
            sol.customers[u].N = sol.customers[u].N[1:max_k_u[u]]
        end
    end
end
    

contract_neighbors! (generic function with 1 method)

In [67]:
function update_model!(LP::OptimizationModel, sol_1::Solution, sol::Solution)
    update_model_demand!(LP, sol_1, sol)

    update_model_time!(LP, sol_1, sol)

    update_model_la!(LP,sol_1,sol)
end

update_model! (generic function with 1 method)

In [68]:
function compute_middle(distance::Float32, max1::Float32, min1::Float32, max2::Float32, min2::Float32)
    minimum_value = min1 + distance
    maximum_value = max1 + distance

    minimum_value = min(minimum_value, max2 -1)
    maximum_value = min(maximum_value, max2 -1)

    if minimum_value == maximum_value
        return minimum_value
    end
    

    


    
end


compute_middle (generic function with 1 method)

In [69]:
function expand_buckets!(MILP::OptimizationModel, sol::Solution)       

    for i in 1:length(sol.demand_nodes)
        for j in sol.demand_nodes[i].forward
            if sol.demand_nodes[i].customer != sol.demand_nodes[j].customer && value(MILP.z_d[(i,j)]) > 0.00001 && j!= length(sol.demand_nodes)
                new_term = sol.customers[sol.demand_nodes[j].customer].demand + sol.demand_nodes[i].min_load - 1
                new_term = min(sol.demand_nodes[j].max_load - 1, floor(new_term))
                new_term = sol.demand_nodes[j].min_load
                #println(" ", sol.demand_nodes[i].max_load, " " , sol.demand_nodes[i].min_load, " ( ", sol.demand_nodes[j].max_load, " " , sol.demand_nodes[j].min_load)
                if !(new_term in sol.customers[sol.demand_nodes[j].customer].W_D) && new_term <= vehicle_capacity
                    push!(sol.customers[sol.demand_nodes[j].customer].W_D, new_term) 
                end
            end
        end
    end
    

    for i in 1:length(sol.time_nodes)
        for j in sol.time_nodes[i].forward
            if sol.time_nodes[i].customer != sol.time_nodes[j].customer && value(MILP.z_t[(i,j)]) > 0.00001 && j!= length(sol.time_nodes)
                new_term = sol.time_nodes[i].min_time + distance_3(sol.time_nodes[i].customer,sol.time_nodes[j].customer)
                #new_term = floor(new_term)
                new_term = min(sol.time_nodes[j].max_time - 1, floor(new_term))
                #new_term -= 1
                if !(new_term in sol.customers[sol.time_nodes[j].customer].W_T)
                    if new_term <= sol.customers[sol.time_nodes[j].customer].time_window[2]
                        if new_term  >= sol.customers[sol.time_nodes[j].customer].time_window[1]
                            push!(sol.customers[sol.time_nodes[j].customer].W_T, new_term)
                        end
                    end
                end
            end
         end   
    end

    # Here we sort all values and remove repeated elements from the list. 
    for u in 1:length(sol.customers) - 1
        sol.customers[u].W_D = sort(unique(sol.customers[u].W_D))
        sol.customers[u].W_T = sort(unique(sol.customers[u].W_T))
    end
    
    update_demand_edges(sol)
    update_time_edges(sol)
end

expand_buckets! (generic function with 1 method)

In [70]:
#### we take a milp solution with dual variables info and create a network of arcs
function merge_buckets!(MILP::OptimizationModel, sol::Solution)
    
    for j in 1:length(sol.demand_nodes)-1
        for i in sol.demand_nodes[j].forward
            if sol.demand_nodes[i].customer == sol.demand_nodes[j].customer && MILP.pi_6j[j] == MILP.pi_6j[i]
                push!(sol.customers[sol.demand_nodes[j].customer].W_D_remove, sol.demand_nodes[j].max_load)
            end
         end   
    end
    for j in 1:length(sol.time_nodes)-1
        for i in sol.time_nodes[j].forward
            if sol.time_nodes[i].customer == sol.time_nodes[j].customer && MILP.pi_6l[j] == MILP.pi_6l[i]
                push!(sol.customers[sol.time_nodes[j].customer].W_T_remove, sol.time_nodes[j].max_time)
            end
         end   
    end
    
    for u in 1:length(sol.customers) - 1
        sol.customers[u].W_D = [w for w in sol.customers[u].W_D if all(abs(w - r) >= 0.005 for r in sol.customers[u].W_D_remove)]
        # Empty W_D_remove after processing
        sol.customers[u].W_D_remove = Float64[]
    end
    
    for u in 1:length(sol.customers) - 1
        sol.customers[u].W_T = [w for w in sol.customers[u].W_T if all(abs(w - r) >= 0.005 for r in sol.customers[u].W_T_remove)]
        # Empty W_D_remove after processing
        sol.customers[u].W_T_remove = Float64[]
        
    end

    #update_demand_edges(sol)
    #update_time_edges(sol)
end


merge_buckets! (generic function with 1 method)

# Algorithm 1,
The main algorithm that is used to solve the problem and it times and prints the total time that is used for the solution of the LP problem
To find the solution times of the gurobi MILP solution it is necessary to read the result directly from gurobi output, which should be done automatically. 

In [75]:
function LA_discretization_algorithm!()
    iter_since_reset = 0
    last_lp_val = -2
    changed = true
    customers =[]
    demand_nodes = []
    time_nodes =[]

    incumbent = Solution(all_demand_nodes, all_time_nodes, all_locations, all_paths)
    
    solution_2 = deepcopy(incumbent)
    LP_model = create_model!(false, solution_2)
    start_time_0 = time()
    unchanged = 0
    LP_time = 0
    while changed
        if iter_since_reset >= 9
            for i in 1:num_locations-1
                solution_2.customers[i].N = solution_2.customers[i].LA
            end
            println("variables get rest ")
        end
        #start_time = time()
        ## debugging no need to create model again. 
        #LP_model = create_model!(false, solution_2)
        
        #end_time = time()
        total_time_nodes = length(solution_2.time_nodes)
        total_demand_nodes = length(solution_2.demand_nodes)
        #println("Time creating model " ,  (end_time - start_time))
        println("Solve")
        println(" Number of time nodes = ", length(solution_2.time_nodes))
        println(" Number of demand nodes = ", length(solution_2.demand_nodes))
        #println(" Number of paths = ", length(solution_2.paths))
        start_time = time()
        solve!(LP_model) ### time the solving of the LP
        end_time = time()
        LP_time += (end_time - start_time)
        println("Time solving model " ,  (end_time - start_time))
        lp_objective = LP_model.objective
        println("objective = ", lp_objective)
        
        if lp_objective > last_lp_val + 1
            unchanged = 0
            solution_1 = deepcopy(solution_2)
            ##incumbent = deepcopy(solution_2)
            ##merge expand and contract only change
            ## the buckets and neighborhood in the solution set
            ## and not the model itself
            
            merge_buckets!(LP_model,solution_2) ## eliminate thresholds
            
            expand_buckets!(LP_model, solution_2) ## add thresholds and update buckets
            contract_neighbors!(LP_model, solution_2) ## reducing the size of N.
            timetoupdate = time()
            
            update_model!(LP_model,solution_1, solution_2)
            timetoupdateend = time()
            #println("time to update model = ", (timetoupdateend - timetoupdate))
            last_lp_val = lp_objective
            iter_since_reset = 0
        else
            solution_1 = deepcopy(solution_2)
            merge_buckets!(LP_model,solution_2)
            expand_buckets!(LP_model, solution_2)
            contract_neighbors!(LP_model, solution_2) ## reducing the size of N.
            update_model!(LP_model,solution_1, solution_2)
        end
        if (lp_objective -last_lp_val) <= 0.001
            unchanged +=1
        else
            unchanged = 0
        end
        unchanged_nodes = total_time_nodes == length(solution_2.time_nodes) && total_demand_nodes == length(solution_2.demand_nodes)
        if unchanged >= 10 #|| unchanged_nodes
            #convert_model_MILP!(LP_model)
            #optimize!(LP_model.model)
            changed = false
        end
        #update_model_test(LP_model,solution_2)
        iter_since_reset += 1
        last_lp_val = lp_objective
    end
    end_time_0 = time()
    #### print the total time spent solving LPs.
    println("Total LP time " ,  LP_time)
    milp = create_MILP!(solution_2)
    optimize!(milp)
end

LA_discretization_algorithm! (generic function with 1 method)

In [74]:
LA_discretization_algorithm!()

Solve
 Number of time nodes = 101
 Number of demand nodes = 201
 Number of paths = 778
Time solving model 0.07088613510131836
objective = 1030.42570332043
time to update model = 0.024403095245361328
Solve
 Number of time nodes = 68
 Number of demand nodes = 118
 Number of paths = 778
Time solving model 0.030848979949951172
objective = 1043.3666761716208
time to update model = 0.014347076416015625
Solve
 Number of time nodes = 69
 Number of demand nodes = 116
 Number of paths = 778
Time solving model 0.031935930252075195
objective = 1043.3666761716208
Solve
 Number of time nodes = 69
 Number of demand nodes = 113
 Number of paths = 778
Time solving model 0.03399300575256348
objective = 1043.3666761716208
Solve
 Number of time nodes = 71
 Number of demand nodes = 114
 Number of paths = 778
Time solving model 0.03251481056213379
objective = 1043.3666761716208
Solve
 Number of time nodes = 69
 Number of demand nodes = 115
 Number of paths = 778
Time solving model 0.03189396858215332
object

In [73]:
num_locations

51